In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Back propagation through time (BPTT)

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>We can "unroll" the RNN into a sequence of layers, one per time step</li>
        <li>In theory: Back Propagation on the unrolled RNN is the same as for a non-Recurrent Network</li>
        <li>In practice: the unrolled RNN is very deep, which causes issues in Back Propagation.
    </ul>
</div>


*Back Propagation Through Time (BPTT)* refers to
- unrolling the RNN computation into a sequence of layers
- performing ordinary Back Propagation in order to update weights

- In a non-Recurrent network:
    - $\W_\llp$, the weights of layer $\ll$, affect only layers $\ll$ and greater.
    - This means the backward flow of the gradient with respect to $\W_\llp$ stops at layer $\ll$.
- In  Recurrent Network:
    - All unrolled "layers" share the *same* weights
    - This means the gradients with respect to shared weight $\W$ must flow backward all the way to the input layer at time $0$.
    

<table>
    <tr>
        <th><center>RNN Loss: Forward pass</center></th>
    </tr>
    <tr>
        <td><img src=images/RNN_layer_loss.png></td>
    </tr>
</table>

<table>
    <tr>
        <th><center>RNN Loss: Backward pass</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_loss_gradient.png"></td>
    </tr>
</table>

The unrolled graph is as deep as the length of $\x^\ip$ ($T^\ip = |\x^\ip|)$ 
- weights can update only after $T^\ip$ input values have been processed,
so training can be slow.
- Vanishing Gradients become a concern for large $T^\ip$
    - Recall from the Vanishing Gradient lecture: magnitude of gradients diminishes from layer $\ll$ to layer $(\ll-1)$ during back propagation

## Calculating gradients with BPTT

### Back propagation: Refresher

The same math that we used to show how to obtain derivatives (for weight updates in Gradient Descent)
will apply to RNN's.

To refresh our memory on notation and results, recall our derivation of back propagation:

Layer $\ll$:
- input/output relation of layer $\ll$ as
$$
\y_{(\ll)} = a_{(\ll)}( f_{(\ll)}( \y_{(\ll-1)}, \W_{(\ll)}) )
$$

for
- activation function $a_\llp$
- weights $\W_\llp$
- $\y_{(\ll-1)}$ are the outputs of the previous layer

- $f_{(\ll)}$ is the function computed by layer $\ll$
    - function of input $\y_{(\ll-1)}$ and weights $\W_\llp$
    - e.g., `Dense`: $f_{(\ll)}( \y_{(\ll-1)}, \W_{(\ll)}) = \y_\llp = \W_\llp \y_{(\ll-1)} + \b_\llp$

**Note** We neglect to add $\b_\llp$ as an argument to $f_\llp$ to simplify notation
- as a convenience we sometimes view $\b_\llp$ as being part of $\W_\llp$

Let 
- $\loss$ denote loss (computed after final layer $L$)
- $\loss'_\llp = \frac{\partial \loss}{\partial y_\llp}$ denote the derivative of $\loss$ with respect to the output of layer $\ll$, i.e., $y_\llp$,
    - refer to as **loss gradient** (at output of layer $\ll$)
    

We showed how to compute
- $\loss'_{(\ll-1)}$ from $\loss'_\ll$ 
    - so that we can continue this process as the previous layer (i.e, *propogate loss gradient backwards*)

and we showed how to compute the weight update
- $\frac{\partial \loss}{\partial W_\llp}$, from $\loss'_\llp$  for $\ll \in [1,L]$

Note that $\y_\llp$ is a function of 
- $\y_{(\ll-1)}$ (the output of the previous layer) 
- and $\W_\llp$, the parameters of layer $\ll$.

We can compute derivatives of $\y_\llp$ with respect to each of its inputs
- $\frac{\partial \y_\llp}{\partial \y_{(\ll-1)}}$
- $\frac{\partial \y_\llp}{\partial \W_\llp}$

Refer to these as **local gradients**

We used the chain rule to obtain the 
- gradient with respect to weights $\W_\llp$, given the loss gradient $\loss'_\llp$ 

$$
\begin{array}[lll] \\
\frac{\partial \loss}{\partial \W_\llp} & = & \frac{\partial \loss}{\partial \y_\llp} \frac{\partial \y_\llp}{\partial \W_\llp} & = & \loss'_\llp \frac{\partial \y_\llp}{\partial \W_\llp}
\end{array}
$$

That is: 
- gradient of $\loss$ with respect to weight $\W_\llp$ 
- is the loss gradient (at current step), multiplied by
- a local gradient (with respect to input  $W_\llp$ )

So we have the information required to update $\W_\llp$ by Gradient Descent.


### BPTT: gradient calculation

Let us adapt these results for the case of a *single layer* RNN
- by "unrolling" this RNN, layer $\ll$ is equated with "time" (of index into input sequence ) $\tt$

Per example loss $\loss^\ip$ is now a per example loss *per time step*
$$
\loss^\ip_\tp
$$
so
$$
\loss^\ip = \sum_{t=1}^T \loss^\ip_\tp
$$

We will focus on the per example loss for a single time $\loss^\ip_\tp$

​
<table>
    <tr>
        <th><center>RNN Loss</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_loss.png"></td>
    </tr>
</table>

In order to compute 
$$
\frac{\partial \loss^\ip}{\partial \W}
$$
we must compute
$$
\frac{\partial \loss^\ip_\tp}{\partial \W}
$$
for each time $t$.

As per regular backprop, we can obtain the loss update by
multiplying the loss gradient by a local gradient

$$
\frac{\partial \loss^\ip_\tp}{\partial \W}
$$

but note that we use unsubscripted $\W$ (rather than $\W_\tp$ because the *same* $\W$ is used at all timesteps.

$$
\frac{\partial \loss^\ip_\tp}{\partial \W} = \loss'_\tp \frac{\partial \y_\tp}{\partial W}
$$

but now 
$$
\frac{\partial \y_\tp}{\partial W}
$$

becomes more complicated, governed by the RNN Update equations

$$
\begin{array}[lll]\\
\h_\tp & = & \phi(\W_{xh}\x_\tp  + \W_{hh}\h_{(t-1)}  + \b_h) \\
\y_\tp & = &  \W_{hy} \h_\tp  + \b_y \\
\end{array}
$$

**Notes**

- In this section we will assume $\phi$ is the identity function to simplify the presentation.

    - There will be no loss of generality.
- Recall that $\W$ is the matrix with embedded sub-matrices $\W_{xh}, \W_{hh}, \W_{hy}$
    - For clarity: we will add subscripts to $\W$ in the derivatives to show which part of $\W$ is the cause.

The equation defining $\y_\tp$
$$
\y_\tp  =   \W_{hy} \h_\tp  + \b_y 
$$

shows that $\y_\tp$ is
- directly depends on  $\W$ (through $\W_{hy}$ )
- *and* indirectly depends on $\W$  through its dependence on $h_\tp$ (which depends on $\W$)

So
$$
\frac{\partial \y^\ip_\tp}{\partial \W} = 
\frac{\partial \y^\ip_\tp}{\partial \W_{hy}}
+
\frac{\partial \y^\ip_\tp}{\partial \h_{(\tt)} } \frac{\partial \h_{(\tt)}}{\partial \W_{hh}}
$$

Let's expand the term
$$
\frac{\partial \h_{(\tt)}}{\partial \W_{hh}}
$$

Recall the recursive definition of $\h_\tp$ 
$$
\h_\tp =  \W_{xh}\x_\tp  + \W_{hh}\h_{(\tt-1)}  + \b_h
$$

$\h_\tp$ depends on $\h_{(\tt-1)}$, which by recursion depends on $\h_{(\tt-2)}$ which $\ldots$ depends on $\h_{(0)}$.
- and all $\h_\tp$ share the *same* $\W_{hh}$.

This means that $\h_\tp$ depends on $\W$ through *each* $\h_{(\tt-k)}$ for $k=1, \ldots, \tt$.
$$
\frac{\partial \h_\tp}{\partial \W_{hh}} = 
\sum_{k=1}^{\tt} { 
 \frac{\partial \h_\tp}{\partial \h_{(\tt -k)}}   \frac{\partial \h_{(\tt -k)}}{\partial \W_{hh}}   
}
$$

So 

$$
\frac{\partial \loss^\ip_\tp}{\partial \W} = \loss'_\tp \frac{\partial \y_\tp}{\partial W}
$$

and
$$\frac{\partial \y^\ip_\tp}{\partial W}$$
*depends* on all time steps from $1$ to $t$.

Thus, the derivative update for $\W$ cannot be computed without the gradient (for each time step $t$)
flowing all the way back to time step $0$.


**Note**

Directly expanding the recursion would show
$$
\frac{\partial \h_\tp}{\partial \h_{(\tt -k)}} = 
\prod_{k'=0}^{k-1} { 
\frac{\partial \h_{(\tt-k')}}{\partial \h_{(\tt -k' -1)}}
} 
$$

It is not necessary now, but will be useful in explaining vanishing/exploding gradients

<table>
    <tr>
        <th><center>RNN Loss Gradient Flow</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_loss_gradient.png"></td>
    </tr>
</table>

## Truncated back progagation through time (TBTT)

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>We "unroll" the RNN into a sequence of  T layers, one per time step</li>
        <li>We compute the loss at each time step t, for t=1 to T.</li>
        <li>The gradient of the loss of time step t flows backward for a limited number of time steps</li>
        <ul>
            <li>Rather than flowing backwards al the way to time step 0</li>
        </ul>
    </ul>
</div>

This is called *Truncated BPTT (TBPTT)*.

The advantage of TBPTT
- more frequent gradient updates

The disadvantage
- the loss at step $t$ won't affect **all** previous time steps (because of truncation)
- the error signal from time $\tt$ does not affect any time steps below $\tt - \tau$.
- this means the RNN has difficulty capturing dependencies longer than $\tau$.

Consider a long piece of text
- The first few words indicate the gender/plurality/age of the subject
- A mis-prediction of, e.g. gender, at word $\tau' > \tau$ causes an error at time step $\tau'$
    - which can't interact with the correct gender in the first few words


Note that there is *no truncation* of the forward pass of the RNN !

Only gradient calculations are truncated.

### TBTT: Variations

There are several ways to truncate the Back Propagation.

We will describe them via a function $f(\tt) = \tt'$
- describes the earliest time
step affecting the gradient of $\loss_\tp$
- that is, it describes the window $\tau$

- Untruncated BPTT
    - $f(\tt) = 0$
- k-truncated BPTT
    - $f(\tt) = \max{}(0, \tt -k)$
- subsequence truncated BPTT
    - $f(\tt) = k * \floor{\tt/k}$

What we refer to as subsequence TBTT seems to be common
- break long sequence $\x^\ip$ into subsequences (chunks) of size $k$
- feed $\x^\ip$ forward as usual
    - at the end of a subsequence: 
        - immediately compute the loss gradients for all time steps within the chunk


# RNN vanishing/exploding gradient problem


<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>A "single-layer RNN that has been unrolled for T time steps</li>
        <ul>
            <li>is mathematically equivalent to a simple NN with T layers</li>
            <li><bold>BUT</bold> all layers share the same weights</li>
        </ul>
        <li>This sharing of weights leads to a problem of Vanishing/Exploding gradients</li>
        <ul>
            <li>Similar to the vanishing gradient problem we derived for simple NN</li>
            <li>but with a different root cause (weight sharing)</li>
        </ul>
    </ul>
</div>

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>Why shared weights are different</li>
        <ul>
            <li>Output <bold>y</bold> at time t is a function of cell state h at time t</li>
            <li>Cell state h at time t is recursively defined</li>
            <ul>
                <li>So it is a function of cell states over all times t' < t as well</li>
                <li>This means the weight update involves a repeated product: (t -t') times</li>
                <li>This product tends to 0 (vanishing) or infinity (explode) as (t -t') increases</li>
            </ul>
            <li>So losses at time step t have difficulty updating gradients for the distant past<</li>
            <li>RNN has difficulty with long-term dependencies</li>   
        </ul>
    </ul>
</div>

Returning to the loss gradient
we encountered the terms

$$\frac{\partial \y^\ip_\tp}{\partial \W}$$

We will focus on the part of $\W$ that is $\W_{hh}$

$$
\frac{\partial \y_\tp}{\partial W_{hh}} = \frac{\partial \y_\tp}{\partial \h_\tp} \frac{{\partial \h_\tp}}{\partial \W_{hh}}
$$

But recursively defined $\h_\tp$ is a function of $\h_{(\tt-1)}, \h_{(\tt-1)}, \ldots, \h_{(1)}$
so

$$
\frac{\partial \y_\tp}{\partial W_{hh}} = \frac{\partial \y_\tp}{\partial \h_\tp}
 \sum_{k=0}^\tt {  \frac{\partial \h_\tp}{\partial \h_{(\tt -k)}} \frac{\partial \h_{(\tt -k)}}{\partial \W_{hh}} }
$$

The summation: $\frac{\partial \h_\tp}{\partial \W_{hh}}$, through all intermediate $\h_{(\tt -k)}$


The problematic term for us is
$$
\frac{\partial \h_\tp}{\partial \h_{(\tt-k)}}
$$

It can be computed by the Chain Rule as
$$
\frac{\partial \h_\tp}{\partial \h_{(\tt-k)}} = \prod_{u=0}^{\tt-1} { \frac{\partial \h_{(\tt -u)}}{\partial \h_{(\tt - u -1) }}}
$$

Each term
$$
\frac{\partial \h_{(\tt -u)}}{\partial \h_{(\tt - u-1)}}
$$
results in a term $\W_{hh}$ so the repeated product compute matrix $\W_{hh}$ raised to the power $k$.



For simplicity,  suppose $\W_{hh}$ were a scalar
- if $\W_{hh} < 1$ then repeatedly multiply $\W_{hh}$ by itself approaches $0$
- if $\W_{hh} > 1$ then repeatedly multiply $\W_{hh}$ by itself approaches $\infty$

In other words:
- as the distance between time steps $\tt$ and $(\tt -k)$ increases
- the gradient (for the weight  update) either vanishes or explodes.

Since this term is used in the update for our weights
- updates  will either be erratic (too big) 
- or
non-existent, hampering learning of weights.

This was not necessarily a problem in non-recurrent networks 
- because each layer had a different
weight matrix.

What an RNN does that helps it be parsimonius in number of parameters
- by sharing the weights across
all time steps
- hurts us in learning.
                                              

For the general case where $\W_{hh}$ is a matrix
- we can show the same resul with the eigenvalues of the matrix

### Controlling exploding gradients by clipping
In theory, we can control the explosion by clipping the gradient $\frac{\partial \loss}{\partial W_i}$.

We are still left with the vanishing gradient problem.

This means that we can't learn long-term dependencies (i.e., too many steps backward).

This will be "solved" by introducing recurrent architectures that address this issue.

In [3]:
print("Done")

Done
